In [1]:
import pandas as pd

In [2]:
movie = pd.read_csv("movie.csv")
rating = pd.read_csv("rating.csv")

In [3]:
movie.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
rating.head(10)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
5,1,112,3.5,2004-09-10 03:09:00
6,1,151,4.0,2004-09-10 03:08:54
7,1,223,4.0,2005-04-02 23:46:13
8,1,253,4.0,2005-04-02 23:35:40
9,1,260,4.0,2005-04-02 23:33:46


In [5]:
movie = movie[~movie["title"].duplicated()]
rating = rating[~rating.duplicated()]

# EDA

In [6]:
df = movie.merge(rating, how="left",on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [7]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Info #####################")
    print(dataframe.info())
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99, 1]).T)

In [8]:
check_df(df)

##################### Shape #####################
(20000219, 6)
##################### Info #####################
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000219 entries, 0 to 20000218
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   title      object 
 2   genres     object 
 3   userId     float64
 4   rating     float64
 5   timestamp  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.0+ GB
None
##################### Types #####################
movieId        int64
title         object
genres        object
userId       float64
rating       float64
timestamp     object
dtype: object
##################### Head #####################
   movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Anim

### Total movie count

In [9]:
df["title"].nunique() 

27262

### Rating counts of a film

In [10]:
rating_counts = pd.DataFrame(df["title"].value_counts())
rating_counts

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Blow Job (1963),1
Zaza (1938),1
"See Here, Private Hargrove (1944)",1
Romance in Manhattan (1935),1


In [11]:
rare_movies = rating_counts[rating_counts["title"] < 1000].index 
rare_movies

Index(['Rosewood (1997)', 'Bear, The (Ours, L') (1988)', 'Ted (2012)',
       'One Night at McCool's (2001)', 'Marked for Death (1990)',
       'Three to Tango (1999)', 'Adam's Rib (1949)',
       'I Now Pronounce You Chuck and Larry (2007)', 'Someone Like You (2001)',
       'Stakeout (1987)',
       ...
       'Le printemps, l'automne et l'amour (1955)',
       'Thank You, Jeeves! (1936)', 'Mabel at the Wheel (1914)',
       'Confidentially Connie (1953)', 'Easy to Love (1953)',
       'Blow Job (1963)', 'Zaza (1938)', 'See Here, Private Hargrove (1944)',
       'Romance in Manhattan (1935)',
       'George Lopez: Tall, Dark & Chicano (2009)'],
      dtype='object', length=24103)

In [12]:
common_movies = df[~df["title"].isin(rare_movies)] 
common_movies.nunique()

movieId          3159
title            3159
genres            578
userId         138493
rating             10
timestamp    13559679
dtype: int64

# User - Movie DF

In [13]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

In [14]:
check_df(user_movie_df)

##################### Shape #####################
(138493, 3159)
##################### Info #####################
<class 'pandas.core.frame.DataFrame'>
Float64Index: 138493 entries, 1.0 to 138493.0
Columns: 3159 entries, 'burbs, The (1989) to ¡Three Amigos! (1986)
dtypes: float64(3159)
memory usage: 3.3 GB
None
##################### Types #####################
title
'burbs, The (1989)                   float64
(500) Days of Summer (2009)          float64
*batteries not included (1987)       float64
...And Justice for All (1979)        float64
10 Things I Hate About You (1999)    float64
                                      ...   
Zulu (1964)                          float64
[REC] (2007)                         float64
eXistenZ (1999)                      float64
xXx (2002)                           float64
¡Three Amigos! (1986)                float64
Length: 3159, dtype: object
##################### Head #####################
title   'burbs, The (1989)  (500) Days of Summer (2009)  \


title
'burbs, The (1989)                   135273
(500) Days of Summer (2009)          132726
*batteries not included (1987)       136763
...And Justice for All (1979)        137357
10 Things I Hate About You (1999)    126880
                                      ...  
Zulu (1964)                          137489
[REC] (2007)                         137442
eXistenZ (1999)                      133195
xXx (2002)                           133951
¡Three Amigos! (1986)                132008
Length: 3159, dtype: int64
##################### Quantiles #####################
                                   0.00   0.05  0.25  0.50  0.75  0.95  0.99  \
title                                                                          
'burbs, The (1989)                  0.5  1.000   2.0   3.0   4.0   5.0   5.0   
(500) Days of Summer (2009)         0.5  2.000   3.5   4.0   4.5   5.0   5.0   
*batteries not included (1987)      0.5  1.500   3.0   3.5   4.0   4.5   5.0   
...And Justice for All (1979)

In [15]:
user_movie_df.tail(10)

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
138484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138485.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138486.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138487.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Random User Selection

In [16]:
random_user = user_movie_df[user_movie_df.index == 108170.0]

### Clear NaN values and creating new list except values which had cleared that includes watched films

In [17]:
movies_watched = random_user.columns[random_user.notna().any().tolist()]
movies_watched

Index(['2001: A Space Odyssey (1968)',
       'Adventures of Priscilla, Queen of the Desert, The (1994)',
       'Akira (1988)', 'Aladdin (1992)', 'Aliens (1986)',
       'Almost Famous (2000)', 'Along Came Polly (2004)', 'Amadeus (1984)',
       'American History X (1998)', 'Any Given Sunday (1999)',
       ...
       'Untouchables, The (1987)', 'Usual Suspects, The (1995)',
       'Wallace & Gromit: A Close Shave (1995)',
       'Wallace & Gromit: The Best of Aardman Animation (1996)',
       'Wallace & Gromit: The Wrong Trousers (1993)',
       'Wedding Singer, The (1998)', 'Who Framed Roger Rabbit? (1988)',
       'Wild at Heart (1990)', 'Willow (1988)', 'X2: X-Men United (2003)'],
      dtype='object', name='title', length=186)

### Watched movies count

In [18]:
len(movies_watched)

186

### Watched movies dataframe that includes all users

In [19]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),...,"Untouchables, The (1987)","Usual Suspects, The (1995)",Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Best of Aardman Animation (1996),Wallace & Gromit: The Wrong Trousers (1993),"Wedding Singer, The (1998)",Who Framed Roger Rabbit? (1988),Wild at Heart (1990),Willow (1988),X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0
2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,3.0,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN


### People counts that watched min 1 same movie with random selected user

In [20]:
movies_watched_df.shape 

(138493, 186)

### How many movies has a user watched?

In [21]:
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count

userId
1.0         53
2.0         11
3.0         47
4.0          5
5.0         16
            ..
138489.0    11
138490.0    10
138491.0     3
138492.0    20
138493.0    57
Length: 138493, dtype: int64

### Turning to dataframe

In [22]:
user_movie_count = user_movie_count.reset_index()
user_movie_count

,userId,0
0,1.0,53
1,2.0,11
2,3.0,47
3,4.0,5
4,5.0,16
...,...,...
138488,138489.0,11
138489,138490.0,10
138490,138491.0,3
138491,138492.0,20


### Changing column names

In [23]:
user_movie_count.columns = ["user_id", "movie_count"]
user_movie_count

,user_id,movie_count
0,1.0,53
1,2.0,11
2,3.0,47
3,4.0,5
4,5.0,16
...,...,...
138488,138489.0,11
138489,138490.0,10
138490,138491.0,3
138491,138492.0,20


### Filtering of people who have watched at least 20 of the movies watched by a randomly selected user

In [24]:
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False).head(10)

,user_id,movie_count
108169,108170.0,186
8404,8405.0,185
118204,118205.0,184
74141,74142.0,183
69792,69793.0,182
118753,118754.0,182
34575,34576.0,181
103653,103654.0,179
29574,29575.0,179
135089,135090.0,178


### Users who have 60% of watching the same movie.

In [25]:
a = len(movies_watched)*60/100           
users_same_movies = user_movie_count[user_movie_count["movie_count"] > a]["user_id"]

users_same_movies

90            91.0
115          116.0
155          156.0
293          294.0
297          298.0
            ...   
138133    138134.0
138207    138208.0
138253    138254.0
138324    138325.0
138410    138411.0
Name: user_id, Length: 2326, dtype: float64

### The number of users watching the same movies according to the evaluation criterias

In [26]:
users_same_movies.count()

2326

### Creating final dataframe

In [27]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],random_user[movies_watched]])
final_df

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),...,"Untouchables, The (1987)","Usual Suspects, The (1995)",Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Best of Aardman Animation (1996),Wallace & Gromit: The Wrong Trousers (1993),"Wedding Singer, The (1998)",Who Framed Roger Rabbit? (1988),Wild at Heart (1990),Willow (1988),X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,
91.0,2.5,4.0,2.5,NaN,4.0,3.0,NaN,3.5,3.5,NaN,...,4.0,3.5,4.0,4.0,4.0,3.5,4.0,NaN,3.5,3.5
116.0,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.5,3.5,...,NaN,4.5,NaN,NaN,3.5,2.0,3.0,NaN,1.0,4.5
156.0,NaN,3.0,NaN,NaN,5.0,5.0,NaN,4.0,NaN,NaN,...,5.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN
294.0,4.5,4.0,4.5,4.0,4.5,3.5,NaN,2.5,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,4.5,3.5
298.0,NaN,NaN,NaN,3.0,5.0,5.0,NaN,NaN,4.0,5.0,...,3.0,5.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138208.0,3.0,3.5,NaN,4.0,3.0,4.0,NaN,3.0,4.0,NaN,...,3.0,4.0,3.0,3.0,NaN,3.0,3.0,2.5,3.0,3.0
138254.0,NaN,NaN,NaN,3.5,4.0,NaN,NaN,NaN,4.5,NaN,...,4.5,4.5,NaN,NaN,NaN,3.0,4.0,NaN,3.5,4.0
138325.0,5.0,NaN,4.5,3.5,4.5,4.5,NaN,5.0,4.5,NaN,...,4.5,5.0,5.0,5.0,5.0,NaN,4.5,NaN,NaN,4.0


### Rewieving final_df

In [28]:
check_df(final_df)

##################### Shape #####################
(2327, 186)
##################### Info #####################
<class 'pandas.core.frame.DataFrame'>
Float64Index: 2327 entries, 91.0 to 108170.0
Columns: 186 entries, 2001: A Space Odyssey (1968) to X2: X-Men United (2003)
dtypes: float64(186)
memory usage: 3.3 MB
None
##################### Types #####################
title
2001: A Space Odyssey (1968)                                float64
Adventures of Priscilla, Queen of the Desert, The (1994)    float64
Akira (1988)                                                float64
Aladdin (1992)                                              float64
Aliens (1986)                                               float64
                                                             ...   
Wedding Singer, The (1998)                                  float64
Who Framed Roger Rabbit? (1988)                             float64
Wild at Heart (1990)                                        float64
Willow (1988)

### Creating user correlation table that shows correlation between users (duplicates dropped.)

In [29]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df

userId    userId  
72838.0   110130.0   -0.581570
33581.0   100618.0   -0.481252
126121.0  89242.0    -0.476220
7232.0    72838.0    -0.472418
104973.0  20011.0    -0.458733
                        ...   
27053.0   19066.0     0.945507
119048.0  54107.0     0.955393
82739.0   54042.0     0.960796
68063.0   32344.0     0.976141
108170.0  108170.0    1.000000
Length: 2704283, dtype: float64

### Converting the above table to dataframe

In [30]:
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.sort_values(by="corr",ascending=False)

,user_id_1,user_id_2,corr
2704282,108170.0,108170.0,1.000000
2704281,68063.0,32344.0,0.976141
2704280,82739.0,54042.0,0.960796
2704279,119048.0,54107.0,0.955393
2704278,27053.0,19066.0,0.945507
...,...,...,...
4,104973.0,20011.0,-0.458733
3,7232.0,72838.0,-0.472418
2,126121.0,89242.0,-0.476220
1,33581.0,100618.0,-0.481252


### Top users which has at least 65% correlation with random selected user

In [31]:
top_users = corr_df[(corr_df["user_id_1"] == 108170.0) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]]. \
    reset_index(drop=True)

### Reviewing top users and creating new dataframe

In [32]:
top_users = top_users.sort_values(by='corr', ascending=False)
top_users

,user_id_2,corr
4,108170.0,1.000000
3,5155.0,0.716406
2,119285.0,0.700081
1,12710.0,0.663332
0,82860.0,0.654646


### Changing column names

In [33]:
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users

,userId,corr
4,108170.0,1.000000
3,5155.0,0.716406
2,119285.0,0.700081
1,12710.0,0.663332
0,82860.0,0.654646


In [34]:
# import main folder for merge and display MovieId and rating informations.
rating = pd.read_csv("rating.csv")

### Top users ratings to movies

In [35]:
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != 108170]
top_users_ratings.head(10)

,userId,corr,movieId,rating
194,5155.0,0.716406,1,3.5
195,5155.0,0.716406,2,3.0
196,5155.0,0.716406,5,3.0
197,5155.0,0.716406,9,3.0
198,5155.0,0.716406,10,4.0
199,5155.0,0.716406,11,3.0
200,5155.0,0.716406,12,3.5
201,5155.0,0.716406,16,3.5
202,5155.0,0.716406,17,2.5
203,5155.0,0.716406,19,4.0


# WEIGHTED AVERAGE RECOMMENDATION SCORE CALCULATION AND RECOMMEND MOVIES

In [36]:
# weighted average score calculation
# cause we've done here is creating new variable to represent mixture of correlation and rating score
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
194,5155.0,0.716406,1,3.5,2.507423
195,5155.0,0.716406,2,3.0,2.149219
196,5155.0,0.716406,5,3.0,2.149219
197,5155.0,0.716406,9,3.0,2.149219
198,5155.0,0.716406,10,4.0,2.865626


In [37]:
# we grouped top_users_ratings dataframe by movieId and values are averaged weighted rating scores
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df

,movieId,weighted_rating
0,1,2.066421
1,2,1.554186
2,3,0.986312
3,5,1.484503
4,6,2.251420
...,...,...
1697,80100,1.989997
1698,80107,0.663332
1699,80139,1.989997
1700,80350,0.663332


In [38]:
check_df(recommendation_df)

##################### Shape #####################
(1702, 2)
##################### Info #####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1702 entries, 0 to 1701
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movieId          1702 non-null   int64  
 1   weighted_rating  1702 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.7 KB
None
##################### Types #####################
movieId              int64
weighted_rating    float64
dtype: object
##################### Head #####################
   movieId  weighted_rating
0        1         2.066421
1        2         1.554186
2        3         0.986312
3        5         1.484503
4        6         2.251420
##################### Tail #####################
      movieId  weighted_rating
1697    80100         1.989997
1698    80107         0.663332
1699    80139         1.989997
1700    80350         0.663332
1701  

In [39]:
recommendation_df[["movieId"]].nunique() 

movieId    1702
dtype: int64

In [40]:
#Let's get the most weighted_rating movies above 2.5
recommendation_df[recommendation_df["weighted_rating"] > 2.149219] 

,movieId,weighted_rating
4,6,2.251420
5,9,2.149219
7,11,2.149219
10,16,2.485452
12,18,2.901848
...,...,...
1691,77658,2.984996
1692,78499,2.653330
1693,78637,2.653330
1694,79008,2.653330


In [41]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 2.149219].sort_values("weighted_rating", \
                                                                                                   ascending=False)
movies_to_be_recommend

,movieId,weighted_rating
1200,5110,3.582032
474,1663,3.582032
830,3098,3.582032
777,2804,3.582032
703,2502,3.582032
...,...,...
1404,6952,2.149219
1409,6992,2.149219
1421,7101,2.149219
1426,7149,2.149219


In [42]:
movie = pd.read_csv('movie.csv')
#Finally we see the first 5 recommended movies for our user
movies_to_be_recommend.merge(movie[["movieId", "title"]]).head(5) 

,movieId,weighted_rating,title
0,5110,3.582032,Super Troopers (2001)
1,1663,3.582032,Stripes (1981)
2,3098,3.582032,"Natural, The (1984)"
3,2804,3.582032,"Christmas Story, A (1983)"
4,2502,3.582032,Office Space (1999)


# Last Film

In [43]:
user = 108170
movie_id = rating[(rating["userId"]==user) & (rating["rating"]==5.0)].sort_values(by="timestamp", \
                                                                                ascending=False)["movieId"][0:1].values[0]  
movie_id

7044

In [44]:
#Let's see movie_id : 7044 is which movie.
def check_movie_id (dataframe,movie_id ):
    movie_name=dataframe[dataframe["movieId"]==movie_id][["title"]].values[0].tolist()
    print(movie_name)

check_movie_id(movie, movie_id) 

['Wild at Heart (1990)']


# Item Based Recommendation


In [45]:
movie_name = movie[movie["title"]=="Wild at Heart (1990)"]
movie_name

,movieId,title,genres
6932,7044,Wild at Heart (1990),Crime|Drama|Mystery|Romance|Thriller


In [46]:
movie_name = "Wild at Heart (1990)"
movie_name = user_movie_df[movie_name]
movies_from_item = user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)
movies_from_item[1:6].index

Index(['My Science Project (1985)', 'Mediterraneo (1991)',
       'Old Man and the Sea, The (1958)',
       'National Lampoon's Senior Trip (1995)', 'Clockwatchers (1997)'],
      dtype='object', name='title')

In [47]:
# Item_based-recommendation
movie_name = "Pulp Fiction (1994)"
movie_name = user_movie_df[movie_name]
movies_from_item = user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)
movies_from_item[1:6].index

Index(['Reservoir Dogs (1992)', 'Kill Bill: Vol. 1 (2003)',
       'Kill Bill: Vol. 2 (2004)', 'Natural Born Killers (1994)',
       'Inglourious Basterds (2009)'],
      dtype='object', name='title')

In [48]:
# Item_based-recommendation
movie_name = "Kill Bill: Vol. 1 (2003)"
movie_name = user_movie_df[movie_name]
movies_from_item = user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)
movies_from_item[1:6].index

Index(['Kill Bill: Vol. 2 (2004)', 'Pulp Fiction (1994)', 'Grindhouse (2007)',
       'Sin City (2005)', 'Inglourious Basterds (2009)'],
      dtype='object', name='title')

# User Based Recommendation


In [49]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df.mean()

weighted_rating    2.081874
dtype: float64

In [50]:
recommendation_df = recommendation_df.reset_index()
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 2.081874].sort_values("weighted_rating", ascending=False)
#movie = pd.read_csv('movie.csv')
movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"].head(5)

0        Dirty Work (1998)
1        Old School (2003)
2    Super Troopers (2001)
3           Stripes (1981)
4      Office Space (1999)
Name: title, dtype: object